In [1]:
###worksfor hypo only as calculated for 'confidence' 

#### need to handle, low mapq and dup

import pandas as pd
from statistics import mean
import time
import numpy as np
from collections import defaultdict
import sys

start_time = time.time()

smfile='/Users/irffanalahi/Research/Research_code/gitignorefolder/MHB_per_read/softreadcounting/testtheidea/try2/try2_efficient/BL14_all_matrixCin_nr0.4_imputed_rowmean.txt_bg_intesectedwith_CD4DMRofBL14atleast.2SM_g1_CD4_3_g2_others_33.txt_othermean.txt'
finalfile='/Users/irffanalahi/Research/Research_code/gitignorefolder/MHB_per_read/softreadcounting/testtheidea/try2/try2_efficient/testempty.txt'

acccol='hypolist'
noACCcol='hyperlist'

smdf=pd.read_csv(smfile,sep="\t",index_col=['position'])
finaldf=pd.read_csv(finalfile,sep="\t")

finaldf=finaldf[~finaldf['assignedcelltype'].isin(['lowmapq', 'DupRead'])]

deltagreaterforpositive=0  ##############>0

deltasmallerfornegative=0 ############<0

smdf.head()

CD4       CD8        nB        NK        Mn      mNeu  \
position                                                                    
chr1:32387132  0.214333  0.507000  0.992000  0.968333  0.968667  0.965667   
chr1:32387376  0.127000  0.633000  0.946667  0.881333  1.000000  0.860000   
chr1:32387403  0.136667  0.869333  0.958333  0.948667  1.000000  0.953667   
chr1:32387409  0.182333  0.969667  0.989667  0.990667  0.982333  0.957333   
chr1:32387417  0.666667  0.957333  0.980333  0.956333  1.000000  0.912667   

                   m8        DC      Eo      Tr  ...  Mn-others  mNeu-others  \
position                                         ...                           
chr1:32387132  0.9420  0.981000  0.9585  0.8695  ...   0.105103     0.101872   
chr1:32387376  0.8300  0.928000  0.7750  0.6930  ...   0.223179     0.072410   
chr1:32387403  0.8635  1.000000  0.9690  0.7770  ...   0.146551     0.096654   
chr1:32387409  0.9900  1.000000  1.0000  0.7570  ...   0.097346     0.070423   
chr1:32387417  1.0000  0.981333  0.9665  0.9300  ...   0.063795    -0.030256   

               m8-others  DC-others  Eo-others  Tr-others  m4-others  \
position                                                               
chr1:32387132   0.076385   0.118385   0.094154  -0.001692   0.052692   
chr1:32387376   0.040103   0.145641  -0.019128  -0.107436  -0.061128   
chr1:32387403  -0.000449   0.146551   0.113167  -0.093603  -0.164141   
chr1:32387409   0.105603   0.116372   0.116372  -0.145321  -0.155013   
chr1:32387417   0.063795   0.043692   0.027718  -0.011590  -0.032590   

               Mg-others  Er-others  mB-others  
position                                        
chr1:32387132   0.138846   0.054846   0.123769  
chr1:32387376   0.113872   0.109564   0.110821  
chr1:32387403   0.077090   0.132551   0.130038  
chr1:32387409   0.093218   0.103449   0.066474  
chr1:32387417   0.023949   0.040103   0.029333  

[5 rows x 28 columns]

In [2]:

allcolumns=smdf.columns.tolist()
allcollength=len(allcolumns)


allct=[item for item in allcolumns if '-others' not in item]
allctlen=len(allct)

if allcollength!=2*allctlen:
    print('wrong all ct inferred. EXITING')
    sys.exit(1)

In [3]:
finaldf[acccol]=finaldf[acccol].apply(lambda x: x[1:-1].split(','))

finaldf[noACCcol]=finaldf[noACCcol].apply(lambda x: x[1:-1].split(','))



In [4]:
finaldf['#CpG']=-1

finaldf['deltabasedfragassignment']='NotAssigned'

score_columns=[]
for ct in allct:
    tempscorecol=ct+'-others'
    finaldf[tempscorecol]='NotAssigned'
    score_columns.append(tempscorecol)
finaldf.head()

ReadName  \
0    mNeu.HWI-ST731_20:4:2313:19955:36893#0   
1  CD4.HWI-ST1450_140:1:2312:17535:63541#12   

                                         allcelltype ucelltype  \
0                       ['CD4', 'CD4', 'CD4', 'CD4']   ['CD4']   
1  ['CD4', 'CD4', 'CD4', 'CD4', 'CD4', 'CD4', 'CD...   ['CD4']   

  assignedcelltype                                         CheckedCpG  \
0      NotAssigned  ['chr11:64490630', 'chr11:64490627', 'chr11:64...   
1              CD4  ['chr1:32387403', 'chr1:32387409', 'chr1:32387...   

            mismatchCpG     notacceptedCpG  \
0  ["'chr11:64490607'"]               ['']   
1                  ['']  ['chr1:32387489']   

                                         acceptedCpG finalacceptedfor  \
0                                               ['']    notdetermined   
1  ['chr1:32387403', 'chr1:32387409', 'chr1:32387...              CD4   

  finalrejectedfor  ...    Mn-others  mNeu-others    m8-others    DC-others  \
0          ['CD4']  ...  NotAssigned  NotAssigned  NotAssigned  NotAssigned   
1    notdetermined  ...  NotAssigned  NotAssigned  NotAssigned  NotAssigned   

     Eo-others    Tr-others    m4-others    Mg-others    Er-others  \
0  NotAssigned  NotAssigned  NotAssigned  NotAssigned  NotAssigned   
1  NotAssigned  NotAssigned  NotAssigned  NotAssigned  NotAssigned   

     mB-others  
0  NotAssigned  
1  NotAssigned  

[2 rows x 36 columns]

In [5]:







def deltacalculation(cpgsfordelta):
    
    
    
    deltafortheseCpGS=cpgsfordelta[score_columns].sum(axis=0)

    return deltafortheseCpGS



def processrow(aindex,arow):
   
    notacceptedCpGList=list(map(eval,arow[noACCcol]))
    acceptedCpGList=list(map(eval,arow[acccol]))
    

    
    
    
    
    accindex=smdf.index.intersection(acceptedCpGList) #acceptedCpGList ##### 
    
    

    
    
    
    
    ACCscore=0
    if len(accindex)>0:
        corresACCcpgs=(smdf.loc[accindex])*-1 
        ACCscore=deltacalculation(corresACCcpgs)
    

  
    
    rejindex=smdf.index.intersection(notacceptedCpGList)  #notacceptedCpGList ##########
    

    
    
    
    REJscore=0
    if len(rejindex)>0:
        corresREJcpgs=smdf.loc[rejindex]
        
        REJscore=deltacalculation(corresREJcpgs)
    

    
    finalscore=ACCscore+REJscore
    


    
    finaldf.loc[index,score_columns]=finalscore #.iloc[0]
 
    
    totalcpgs=len(accindex)+len(rejindex)
    
    finaldf.loc[index,'#CpG']=totalcpgs

In [6]:
for index, row in finaldf.iterrows():
    

    processrow(index, row)

In [7]:
finaldf.head()

ReadName  \
0    mNeu.HWI-ST731_20:4:2313:19955:36893#0   
1  CD4.HWI-ST1450_140:1:2312:17535:63541#12   

                                         allcelltype ucelltype  \
0                       ['CD4', 'CD4', 'CD4', 'CD4']   ['CD4']   
1  ['CD4', 'CD4', 'CD4', 'CD4', 'CD4', 'CD4', 'CD...   ['CD4']   

  assignedcelltype                                         CheckedCpG  \
0      NotAssigned  ['chr11:64490630', 'chr11:64490627', 'chr11:64...   
1              CD4  ['chr1:32387403', 'chr1:32387409', 'chr1:32387...   

            mismatchCpG     notacceptedCpG  \
0  ["'chr11:64490607'"]               ['']   
1                  ['']  ['chr1:32387489']   

                                         acceptedCpG finalacceptedfor  \
0                                               ['']    notdetermined   
1  ['chr1:32387403', 'chr1:32387409', 'chr1:32387...              CD4   

  finalrejectedfor  ... Mn-others mNeu-others  m8-others DC-others  Eo-others  \
0          ['CD4']  ... -0.410359   -0.164462   0.400026 -0.276103  -0.120667   
1    notdetermined  ... -0.628654   -0.318141  -0.383295 -0.654859  -0.456526   

  Tr-others  m4-others  Mg-others Er-others  mB-others  
0 -0.112051   0.523872  -0.570282 -0.450744  -0.207179  
1  0.340936    1.00432  -0.430141 -0.556679  -0.532808  

[2 rows x 36 columns]

In [8]:

finaldf=finaldf[finaldf['#CpG']>0]

finaldf.head()

ReadName  \
0    mNeu.HWI-ST731_20:4:2313:19955:36893#0   
1  CD4.HWI-ST1450_140:1:2312:17535:63541#12   

                                         allcelltype ucelltype  \
0                       ['CD4', 'CD4', 'CD4', 'CD4']   ['CD4']   
1  ['CD4', 'CD4', 'CD4', 'CD4', 'CD4', 'CD4', 'CD...   ['CD4']   

  assignedcelltype                                         CheckedCpG  \
0      NotAssigned  ['chr11:64490630', 'chr11:64490627', 'chr11:64...   
1              CD4  ['chr1:32387403', 'chr1:32387409', 'chr1:32387...   

            mismatchCpG     notacceptedCpG  \
0  ["'chr11:64490607'"]               ['']   
1                  ['']  ['chr1:32387489']   

                                         acceptedCpG finalacceptedfor  \
0                                               ['']    notdetermined   
1  ['chr1:32387403', 'chr1:32387409', 'chr1:32387...              CD4   

  finalrejectedfor  ... Mn-others mNeu-others  m8-others DC-others  Eo-others  \
0          ['CD4']  ... -0.410359   -0.164462   0.400026 -0.276103  -0.120667   
1    notdetermined  ... -0.628654   -0.318141  -0.383295 -0.654859  -0.456526   

  Tr-others  m4-others  Mg-others Er-others  mB-others  
0 -0.112051   0.523872  -0.570282 -0.450744  -0.207179  
1  0.340936    1.00432  -0.430141 -0.556679  -0.532808  

[2 rows x 36 columns]

In [9]:

ctpropdict= defaultdict(list)


for score in score_columns:
    

    
    posscore=finaldf.loc[finaldf[score]>deltagreaterforpositive,score]
    
    negscore=finaldf.loc[finaldf[score]<deltasmallerfornegative,score]
    
    
    
    poscpg=(finaldf.loc[finaldf[score]>deltagreaterforpositive,'#CpG']).sum()
    negcpg=(finaldf.loc[finaldf[score]<deltasmallerfornegative,'#CpG']).sum()
    
    
    poslen=len(posscore)
    neglen=len(negscore)
    
    posscoreSUM=sum(posscore)
    negscoreSUM=sum(negscore)
    
    
    
    ct_posnegscore=posscoreSUM/(posscoreSUM+abs(negscoreSUM))
    
    
    if poscpg>0:
        ct_posscorebyposcpg=posscoreSUM/poscpg
    else:
        ct_posscorebyposcpg=0
    
    ct_posscorebyallcpg=posscoreSUM/(poscpg+negcpg)
    
    ct_fragscore=poslen/(poslen+neglen)
    
    
    ctname=score.replace('-others','')
    
    ctpropdict[ctname+"_posnegscore"].append(ct_posnegscore)
    ctpropdict[ctname+"_posscorebyposcpg"].append(ct_posscorebyposcpg)
    ctpropdict[ctname+"_posscorebyallcpg"].append(ct_posscorebyallcpg)
    ctpropdict[ctname+"_fragscore"].append(ct_fragscore)
    
    
    
    
    
    

In [10]:
ctpropdf=pd.DataFrame.from_dict(ctpropdict)
ctpropdf.to_csv(finalfile+"_softRD_result.txt",sep="\t",index=False)

In [11]:
finaldf.to_csv(finalfile+"_softRD.txt",sep="\t",index=False)